In [32]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np

In [34]:
# Set the working directory
working_directory = os.getcwd()
print(f"Current working directory: {working_directory}")

Current working directory: /Users/buboyencarnacion


In [36]:
# Load cleaned datasets
customers_df = pd.read_csv(os.path.join(working_directory, 'cleaned_customers_data.csv'))
products_df = pd.read_csv(os.path.join(working_directory, 'cleaned_products_data.csv'))
transactions_df = pd.read_csv(os.path.join(working_directory, 'cleaned_transactions_data.csv'))


In [38]:
# Convert Product_ID to string in both datasets
products_df['Product_ID'] = products_df['Product_ID'].astype(str)
transactions_df['Product_ID'] = transactions_df['Product_ID'].astype(str)

In [40]:
# Convert Company_ID to string in both datasets
customers_df['Company_ID'] = customers_df['Company_ID'].astype(str)
transactions_df['Company_ID'] = transactions_df['Company_ID'].astype(str)

In [42]:
# Ensure no missing values in key columns
assert not transactions_df[['Product_ID', 'Company_ID']].isnull().any().any()
assert not customers_df[['Company_ID']].isnull().any().any()
assert not products_df[['Product_ID']].isnull().any().any()

In [44]:
# Step 1: Merge Transactions with Products on 'Product_ID'
merged_df = transactions_df.merge(products_df, on='Product_ID')

In [46]:
# Step 2: Merge the result with Customers on 'Company_ID'
merged_df = merged_df.merge(customers_df, on='Company_ID')


In [48]:
# Drop unnecessary columns (optional)
merged_df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')


In [50]:
# Check for missing values
missing_values = merged_df.isnull().sum()
print("\nMissing Values in Merged DataFrame:")
print(missing_values[missing_values > 0])


Missing Values in Merged DataFrame:
Transaction_Date    6199
dtype: int64


In [52]:
### Merged Dataset Preprocessing ###

# Fill missing values
numerical_columns = merged_df.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_columns:
    merged_df[col].fillna(merged_df[col].median(), inplace=True)

categorical_columns = merged_df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    merged_df[col].fillna(merged_df[col].mode()[0], inplace=True)

/var/folders/yw/ghyqsycn5z34j1zd_gpjk9bw0000gn/T/ipykernel_1091/1424380386.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df[col].fillna(merged_df[col].median(), inplace=True)
/var/folders/yw/ghyqsycn5z34j1zd_gpjk9bw0000gn/T/ipykernel_1091/1424380386.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [54]:
# Label Encoding Categorical Features
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    merged_df[col] = le.fit_transform(merged_df[col])
    label_encoders[col] = le


In [58]:
# Scaling Numerical Features
scaler = StandardScaler()
numerical_features = ['Product_Price', 'Quantity', 'Total_Cost', 'Company_Profit']
numerical_features = [col for col in numerical_features if col in merged_df.columns]

if numerical_features:
    merged_df[numerical_features] = scaler.fit_transform(merged_df[numerical_features])

In [60]:
### Define Features and Target ###

# Define X (features) and y (target)
X = merged_df.drop(columns=['Product_Name', 'Transaction_Date'])
y = merged_df['Product_Name']

# Ensure all features are numeric
print("\nData types of features:")
print(X.dtypes)


Data types of features:
Transaction_ID            float64
Company_ID                  int64
Product_ID                  int64
Quantity                  float64
Product_Price_x           float64
Total_Cost                float64
Recency                   float64
Purchase_Frequency        float64
Total_Spending            float64
Product_Price_y           float64
Price_Range                 int64
Company_Name                int64
Company_Profit            float64
Address                     int64
Profitability_Category      int64
Region                      int64
dtype: object


In [62]:
### Split Data ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
### Model 1: Multinomial Logistic Regression ###
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
logistic_model.fit(X_train, y_train)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [66]:
# Predictions
logistic_predictions = logistic_model.predict(X_test)

In [68]:
# Evaluation Metrics
logistic_accuracy = accuracy_score(y_test, logistic_predictions)
logistic_report = classification_report(y_test, logistic_predictions, output_dict=True)

print("\nMultinomial Logistic Regression Results:")
print(f"Accuracy: {logistic_accuracy}")
print(classification_report(y_test, logistic_predictions))


Multinomial Logistic Regression Results:
Accuracy: 0.3263094521372667
              precision    recall  f1-score   support

           0       0.26      0.27      0.27        67
           1       0.32      0.19      0.24        97
           2       0.12      0.14      0.13        69
           3       0.12      0.04      0.06        78
           4       0.00      0.00      0.00        86
           5       0.29      0.41      0.34        85
           6       0.62      0.79      0.69        80
           7       0.44      0.34      0.38        83
           8       0.25      0.17      0.20        78
           9       0.14      0.06      0.08        86
          10       0.25      0.04      0.07        72
          11       0.24      0.40      0.30        91
          12       0.22      0.40      0.29        72
          13       0.14      0.05      0.08        92
          14       0.20      0.46      0.28        81
          15       0.00      0.00      0.00        90
          

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa

In [70]:
### Model 2: Random Forest Classifier ###
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [72]:
# Predictions
rf_predictions = random_forest_model.predict(X_test)

In [74]:
# Evaluation Metrics
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_report = classification_report(y_test, rf_predictions, output_dict=True)

print("\nRandom Forest Classifier Results:")
print(f"Accuracy: {rf_accuracy}")
print(classification_report(y_test, rf_predictions))


Random Forest Classifier Results:
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        67
           1       1.00      1.00      1.00        97
           2       1.00      1.00      1.00        69
           3       1.00      1.00      1.00        78
           4       1.00      1.00      1.00        86
           5       1.00      1.00      1.00        85
           6       1.00      1.00      1.00        80
           7       1.00      1.00      1.00        83
           8       1.00      1.00      1.00        78
           9       1.00      1.00      1.00        86
          10       1.00      1.00      1.00        72
          11       1.00      1.00      1.00        91
          12       1.00      1.00      1.00        72
          13       1.00      1.00      1.00        92
          14       1.00      1.00      1.00        81
          15       1.00      1.00      1.00        90
          16       1.00      1.0

In [76]:
### Model Comparison Matrix ###
comparison_table = pd.DataFrame({
    "Model": ["Multinomial Logistic Regression", "Random Forest Classifier"],
    "Accuracy": [logistic_accuracy, rf_accuracy],
    "Precision (Weighted Avg)": [
        logistic_report["weighted avg"]["precision"],
        rf_report["weighted avg"]["precision"]
    ],
    "Recall (Weighted Avg)": [
        logistic_report["weighted avg"]["recall"],
        rf_report["weighted avg"]["recall"]
    ],
    "F1-Score (Weighted Avg)": [
        logistic_report["weighted avg"]["f1-score"],
        rf_report["weighted avg"]["f1-score"]
    ]
})

In [78]:
# Save comparison table to CSV
comparison_table.to_csv("model_comparison.csv", index=False)

In [80]:
# Print the comparison table
print("Model Comparison Table:")
print(comparison_table)

Model Comparison Table:
                             Model  Accuracy  Precision (Weighted Avg)  \
0  Multinomial Logistic Regression  0.326309                  0.263638   
1         Random Forest Classifier  1.000000                  1.000000   

   Recall (Weighted Avg)  F1-Score (Weighted Avg)  
0               0.326309                 0.272098  
1               1.000000                 1.000000  


In [82]:
### Predict on the Same Sample Data ###
sample_data = X_test.iloc[:12]  # Predicting 12 datapoints

logistic_predictions = logistic_model.predict(sample_data)
rf_predictions = random_forest_model.predict(sample_data)


In [84]:
# Get original label classes
product_classes = label_encoders["Product_Name"].classes_

# Function to safely decode labels
def safe_inverse_transform(predictions, encoder_classes):
    return np.array([
        encoder_classes[pred] if pred < len(encoder_classes) else "Unknown"
        for pred in predictions
    ])

In [86]:
# Convert predictions back to original labels
logistic_predictions_original = safe_inverse_transform(logistic_predictions, product_classes)
rf_predictions_original = safe_inverse_transform(rf_predictions, product_classes)
actual_original = safe_inverse_transform(y_test.loc[sample_data.index].values, product_classes)

In [88]:
# Combine predictions into a single DataFrame
comparison_df = pd.DataFrame({
    'Sample Index': sample_data.index,
    'Actual': actual_original,
    'Logistic Regression Prediction': logistic_predictions_original,
    'Random Forest Prediction': rf_predictions_original
})

In [90]:
# Display the comparison table
print("\nComparison of Predictions Side by Side:")
print(comparison_df)


Comparison of Predictions Side by Side:
    Sample Index                        Actual Logistic Regression Prediction  \
0           5758        CustomerScope Insights         DashSync Analytics Hub   
1           4987            NexGen Segmentator             SegmentX Targeting   
2           4322           SalesSync Optimizer               BudgetMaster Pro   
3           2821            SegmentX Targeting             SegmentX Targeting   
4           4696           ForecastXcelerator    FinSphere Intelligence Suite   
5           1858            SegmentX Targeting             SegmentX Targeting   
6           1820           ForecastXcelerator    FinSphere Intelligence Suite   
7           7877  InsightMax Insights Platform         DashSync Analytics Hub   
8           6676           OptiFlow Automation            OptiFlow Automation   
9           7070            NexGen Segmentator             SegmentX Targeting   
10           982                     Product 9                      

In [92]:
# Ensure sample_input has the same feature names and order as training data
sample_input = pd.DataFrame({
    "Quantity": [2],
    "Total_Cost": [6000],
    "Company_Name": label_encoders["Company_Name"].transform(["Sky  Industries  29"]),
    "Product_Price": [12000],
    "Company_Profit": [100000],
})